In [ ]:
#Importig all the libraries

import os
import gdown
import tensorflow as tf
from deepface.commons import functions
from deepface.commons.logger import Logger
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
        Convolution2D,
        ZeroPadding2D,
        MaxPooling2D,
        Flatten,
        Dropout,
        Activation,
        Lambda,
)

In [ ]:
#VGG-Face Model
def loadModel():
    model = Sequential()
    model.add(data_augmentation)

    model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
    model.add(Convolution2D(64, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Convolution2D(4096, (7, 7), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation("softmax"))
    model.build((None, 224, 224, 3)) 

    model.load_weights(os.path.join("/kaggle/input","aaazzzz", "vgg_face_weights.h5"))

    model_output = Flatten()(model.layers[-5].output)
    model_output = Lambda(lambda x: K.l2_normalize(x, axis=1), name="norm_layer")(
        model_output
    )

    return Model(inputs=model.input, outputs=model_output)


data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomTranslation(0.1, 0.1, fill_mode="nearest", interpolation="bilinear")
    ]
)
#Pulling the celebrity faces dataset

base_path = '/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset'
folders = os.listdir(base_path)[:10]
df = pa.DataFrame(
    [
        (os.path.join(base_path, folder, picture), folder[5:])
        for folder in folders
        for picture in os.listdir(os.path.join(base_path, folder))
    ],
    columns=['path', 'celebrity']
).assign(celebrity_encoded=lambda x: LabelEncoder().fit_transform(x['celebrity']))
folds = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(X=df['path'], y=df['celebrity_encoded']))

import cv2
from matplotlib import pyplot as plt
from xgboost import XGBClassifier

def classification1 (train_embedding, test_embedding, train_labels, test_labels):
    scaler = StandardScaler()
    train_embedding = scaler.fit_transform(train_embedding)
    test_embedding = scaler.transform(test_embedding)
    pca = PCA(n_components=128)
    train_embedding = pca.fit_transform(train_embedding)
    test_embedding = pca.transform(test_embedding)
    re = RandomForestClassifier(n_jobs = -1)
    re.fit(train_embedding, train_labels)
    predict  = re.predict(test_embedding)
    return accuracy_score(test_labels, predict)

def classification2 (train_embedding, test_embedding, train_labels, test_labels):
    scaler = StandardScaler()
    train_embedding = scaler.fit_transform(train_embedding)
    test_embedding = scaler.transform(test_embedding)
    pca = PCA(n_components=128)
    train_embedding = pca.fit_transform(train_embedding)
    test_embedding = pca.transform(test_embedding)
    clf = SVC(C=5., gamma=0.001)
    clf.fit(train_embedding, train_labels)
    predict = clf.predict(test_embedding)
    return accuracy_score(test_labels, predict)

def classification3 (train_embedding, test_embedding, train_labels, test_labels):
    re = RandomForestClassifier(n_jobs = -1)
    re.fit(train_embedding, train_labels)
    predict  = re.predict(test_embedding)
    return accuracy_score(test_labels, predict)

def classification4 (train_embedding, test_embedding, train_labels, test_labels):
    xgb = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
    xgb.fit(train_embedding, train_labels)
    predict = xgb.predict(test_embedding)
    return accuracy_score(test_labels, predict)
    
import cv2
import matplotlib.pyplot as plt

def img_to_face(img):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if len(faces) == 0:
        return None
    x, y, w, h = faces[0]
    face_region = img[y:y+h, x:x+w]
    face_region_rgb = cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB)
    return face_region_rgb

def test_model(model_id):
    print("Model - > VG" )
    answer = []
    fold_number = 0
    for fold in folds:
        print("Fold -> ", fold_number)
        train = df.iloc[fold[0]]
        test = df.iloc[fold[1]]
        train_labels = train["celebrity_encoded"]
        test_labels = test["celebrity_encoded"]
        
        train_embedding=[]
        test_embedding = []
        
        model = loadModel()

        for counter, (index, img) in enumerate(train.iterrows()):
            print(counter)
            
            imgg = img_to_face(cv2.imread(img['path']))
            if imgg is None:
                imgg = cv2.imread(img['path'])

            imgg = cv2.resize(imgg, (224, 224))
            imgg = np.expand_dims(imgg, axis=0)
            imgg = imgg.astype('float64') / 255.0 
            train_embedding.append( model.predict(imgg)[0, :])

        print(len(test))
        for counter, (index, img) in enumerate(test.iterrows()):
            print(counter)
            
            imgg = img_to_face(cv2.imread(img['path']))
            if imgg is None:
                imgg = cv2.imread(img['path'])
            imgg = cv2.resize(imgg, (224, 224))
            imgg = imgg.astype('float64') / 255.0 
            imgg = np.expand_dims(imgg, axis=0) 
            
            test_embedding.append( model.predict(imgg)[0, :])
        
        answer.append([classification1(train_embedding, test_embedding, train_labels, test_labels), classification2(train_embedding, test_embedding, train_labels, test_labels),classification3(train_embedding, test_embedding, train_labels, test_labels),classification4(train_embedding, test_embedding, train_labels, test_labels)])
        print("For this fold -> ", answer[fold_number])
        fold_number = fold_number + 1
    res1 = 0
    res2 = 0
    res3=0
    res4=0
    
    print(res1, res2, res3, res4)
    for i in range(fold_number):
        res1 += answer[i][0]
        res2 += answer[i][1]
        res3 += answer[i][2]
        res4 += answer[i][3]
    return (res1 / fold_number, res2 / fold_number, res3/ fold_number, res4/foldernumber)

def models_test():
    for i in range(0,8):
        ress = test_model(i)
        print("For model -> ", "vg", " The classification accuracy with rf + pca +ss -> " , ress[0], " accuracy with svc + pca + ss -> ", ress[1], " accuracy with normal rf -> ", ress[2] , ' XGBOST=> ' , ress[3])
    
models_test()

!PIP INSTALL ULTRALYTICS


I have used the following methods.

I have implemented YOLOV8 model stemmed from this Keras example[1,2],
Used tf.data for input pipeline,
I created txt file parser function,
My Another Segmentation Projects
Lung Segmentation UNet w/SeparableConv (Dice:0.93)
Brain tumor/anomaly segmentation with Unet using TPU
Brain tissue segmentation with Unet using TPU (Dice: 0.88)
References
https://keras.io/examples/vision/yolov8/
Reis, D., Kupec, J., Hong, J., & Daoudi, A. (2023). Real-Time Flying Object Detection with YOLOv8 (Version 1). arXiv. https://doi.org/10.48550/ARXIV.2305.09972
# Importing dependencies

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import Callback
import keras_cv


BATCH_SIZE = 4
GLOBAL_CLIPNORM = 10.0

AUTO = tf.data.AUTOTUNE
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using TensorFlow backend
Preprocessing
# a function for converting txt file to list
def parse_txt_annot(img_path, txt_path):
    img = cv2.imread(img_path)
    w = int(img.shape[0])
    h = int(img.shape[1])

    file_label = open(txt_path, "r")
    lines = file_label.read().split('\n')
    
    boxes = []
    classes = []
    
    for i in range(0, int(len(lines))):
        objbud=lines[i].split(' ')
        class_ = int(objbud[0])
        
        x1 = float(objbud[1])
        y1 = float(objbud[2])
        w1 = float(objbud[3])
        h1 = float(objbud[4])
        
        xmin = int((x1*w) - (w1*w)/2.0)
        ymin = int((y1*h) - (h1*h)/2.0)
        xmax = int((x1*w) + (w1*w)/2.0)
        ymax = int((y1*h) + (h1*h)/2.0)
    
        boxes.append([xmin ,ymin ,xmax ,ymax])
        classes.append(class_)
    
    return img_path, classes, boxes


# a function for creating file paths list 
def create_paths_list(path):
    full_path = []
    images = sorted(os.listdir(path))
    
    for i in images:
        full_path.append(os.path.join(path, i))
        
    return full_path


class_ids = ['apart_parking', 'blackbox', 'disabled_parking', 'etc', 'highpass', 'navigator']
class_mapping = dict(zip(range(len(class_ids)), class_ids))
# a function for creating a dict format of files
def creating_files(img_files_paths, annot_files_paths):
    
    img_files = create_paths_list(img_files_paths)
    annot_files = create_paths_list(annot_files_paths)
    
    image_paths = []
    bbox = []
    classes = []
    
    for i in range(0,len(img_files)):
        image_path_, classes_, bbox_ = parse_txt_annot(img_files[i], annot_files[i])
        image_paths.append(image_path_)
        bbox.append(bbox_)
        classes.append(classes_)
        
    image_paths = tf.ragged.constant(image_paths)
    bbox = tf.ragged.constant(bbox)
    classes = tf.ragged.constant(classes)
    
    return image_paths, classes, bbox
# applying functions
train_img_paths, train_classes, train_bboxes = creating_files('/kaggle/input/disabled-people-sign-detection/disabled_sign_detection/train/images', 
                                                              '/kaggle/input/disabled-people-sign-detection/disabled_sign_detection/train/labels')

valid_img_paths, valid_classes, valid_bboxes = creating_files('/kaggle/input/disabled-people-sign-detection/disabled_sign_detection/valid/images', 
                                                              '/kaggle/input/disabled-people-sign-detection/disabled_sign_detection/valid/labels')

test_img_paths, test_classes, test_bboxes = creating_files('/kaggle/input/disabled-people-sign-detection/disabled_sign_detection/test/images', 
                                                           '/kaggle/input/disabled-people-sign-detection/disabled_sign_detection/test/labels')
Creating Datasets
# reading and resizing images
def img_preprocessing(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.cast(img, tf.float32) 
    
    return img



resizing = keras_cv.layers.JitteredResize(
    target_size=(640, 640),
    scale_factor=(0.75, 1.3),
    bounding_box_format="xyxy")

# loading dataset
def load_ds(img_paths, classes, bbox):
    img = img_preprocessing(img_paths)

    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox }
    
    return {"images": img, "bounding_boxes": bounding_boxes}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]
# Creating dataset loaders and tf.datasets
train_loader = tf.data.Dataset.from_tensor_slices((train_img_paths, train_classes, train_bboxes))
train_dataset = (train_loader
                 .map(load_ds, num_parallel_calls = AUTO)
                 .shuffle(BATCH_SIZE*10)
                 .ragged_batch(BATCH_SIZE, drop_remainder = True)
                 .map(resizing, num_parallel_calls = AUTO)
                 .map(dict_to_tuple, num_parallel_calls = AUTO)
                 .prefetch(AUTO))


valid_loader = tf.data.Dataset.from_tensor_slices((valid_img_paths, valid_classes, valid_bboxes))
valid_dataset = (valid_loader
                 .map(load_ds, num_parallel_calls = AUTO)
                 .ragged_batch(BATCH_SIZE, drop_remainder = True)
                 .map(resizing, num_parallel_calls = AUTO)
                 .map(dict_to_tuple, num_parallel_calls = AUTO)
                 .prefetch(AUTO))


test_loader = tf.data.Dataset.from_tensor_slices((test_img_paths, test_classes, test_bboxes))
test_dataset = (test_loader
                .map(load_ds, num_parallel_calls = AUTO)
                .ragged_batch(BATCH_SIZE, drop_remainder = True)
                .map(resizing, num_parallel_calls = AUTO)
                .map(dict_to_tuple, num_parallel_calls = AUTO)
                .prefetch(AUTO))
# a function to visualize samples from a dataset
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs[0], inputs[1]
    
    keras_cv.visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale = 8,
        font_scale = 0.8,
        line_thickness=2,
        dpi = 100,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
        true_color = (192, 57, 43))
# examples images and annotations from training daatset
visualize_dataset(train_dataset, bounding_box_format="xyxy", value_range=(0, 255), rows=2, cols=2)

YOLO V8 Model
# creating pre-trained model backbone with coco weights
backbone = keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_m_backbone_coco")
Attaching 'config.json' from model 'keras/yolov8/keras/yolo_v8_m_backbone_coco/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/yolov8/keras/yolo_v8_m_backbone_coco/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/yolov8/keras/yolo_v8_m_backbone_coco/2' to your Kaggle notebook...
/opt/conda/lib/python3.10/site-packages/keras_cv/src/models/backbones/backbone.py:44: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/opt/conda/lib/python3.10/site-packages/keras_cv/src/models/backbones/backbone.py:44: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
YOLOV8_model = keras_cv.models.YOLOV8Detector(num_classes=len(class_mapping),
                                              bounding_box_format="xyxy",
                                              backbone=backbone, fpn_depth=1 )

optimizer = AdamW(learning_rate=0.001, weight_decay=0.004, global_clipnorm = GLOBAL_CLIPNORM)

YOLOV8_model.compile(optimizer = optimizer, classification_loss = 'binary_crossentropy', box_loss = 'ciou')
Training 
hist = YOLOV8_model.fit(train_dataset, validation_data = valid_dataset,  epochs = 80 )
Training Results, Evaluation
fig, axs = plt.subplots(1,3, figsize = (18,5), dpi = 130)

axs[0].grid(linestyle="dashdot")
axs[0].set_title("Loss")
axs[0].plot(hist.history['loss'][1:])
axs[0].plot(hist.history['val_loss'][1:])
axs[0].legend(["train", "validataion"])

axs[1].grid(linestyle="dashdot")
axs[1].set_title("Box Loss")
axs[1].plot(hist.history['box_loss'])
axs[1].plot(hist.history['val_box_loss'])
axs[1].legend(["train",  "validataion"])

axs[2].grid(linestyle="dashdot")
axs[2].set_title("Class Loss")
axs[2].plot(hist.history['class_loss'][1:])
axs[2].plot(hist.history['val_class_loss'][1:])
axs[2].legend(["train",  "validataion"])
<matplotlib.legend.Legend at 0x7cbe84c1afe0>

Test Predictions
def visualize_predict_detections(model, dataset, bounding_box_format):
    images, y_true = next(iter(dataset.take(1)))
        
    y_pred = model.predict(images)
    y_pred = keras_cv.bounding_box.to_ragged(y_pred)
    
